In [1]:
import pandas as pd 
import time as t
from lxml import html  
import requests
import json
import numpy as np
from timeit import default_timer as timer
from itertools import cycle
import string

In [2]:
import math
import copy

In [3]:
%load_ext autoreload
%autoreload 2

In [4]:
import sys
sys.path.append('../../')

In [5]:
cd ~/Documents/GitHub/me.nu/menu_read

/Users/michaelsprintson/Documents/GitHub/me.nu/menu_read


In [6]:
food = True

In [7]:
if food:
    respage = 'https://www.yelp.com/biz/mala-sichuan-bistro-houston-3'
    switch = 'food'
    import ocr_food as ocr
    import just_read_food as jr
    pid = 'ChIJNc4K5cTCQIYRe9OyIN7DcGE'
else:
    respage = 'https://www.yelp.com/biz/sharetea-houston-2'
    switch = 'tea'
    import ocr_tea as ocr
    import just_read_tea as jr
    pid = 'ChIJjfzjCM_CQIYRPA546CYaE4A'

# OCR FUNCTIONALITY TEST

In [8]:
# pip install --upgrade google-cloud-vision

In [9]:
# create dictionary
pic_loc = 'ocr/menupictures/pic7.jpg'
test_file_name = 'mala7'
ocr.detect_text(pic_loc, test_file_name)

In [10]:
jr.final_dump("ocr/menu_tests/mala7.txt", "pref_sample.txt", True, "mala7")

32 32
H1 Top Notch Pot the Outlaws 31.95
H2 Sauerkraut Fish and Lamb Pot 24.95
H3 Country Style Blood Curd Dish 14.95
H4 Beef Pot 13.95
H5 and Crispy Chicken 12.95
H6 Duck Stew 12.95
H7 Konjac Beef 12.95
H8 Lotus Tofu 11.95
H9 Three Pepper Beaten Duck 13.95
H10 Tea Smoked Duck 13.95
H11 Four Joy Lion Head 12.95
H12 Fragrant Ribs 12.95
F1 Northern Boiling Fish 20.95
F2 Sauerkraut Fish Pot 20.95
F3 Chengdu Style Pot Roasted Tilapia 18.95
F4 Mala Pot Roasted Tilapia 18.95
F5 Bean Curd Tilapia Stew 18.95
F7 Pot Roasted Black Bean Tilapia 18.95
F12 Steamed Tilapia 18.95
F9 Sweet and Sour Crispy Tilapia 18.95
F10 Scallion Pot Roasted Tilapia 18.95
B1 Water Boiled Beef 13.95
B2 Cumin Beef 13.95
B3 Black Pepper Beef 13.95
B4 Mala Beef Tofu 13.95
B5 Mongolian Beef 13.95
B6 Crispy Mala Beef 13.95
B7 Baby Pepper Black Bean Beef 13.95
B8 Baby Pepper and Shredded Beef 13.95
B9 Satay Beef 13.95
B12 Sauteed Lamb with Veggies 16.95
B11 Cumin Lamb 16.95


{'H1 Top Notch Pot the Outlaws': 31.95,
 'H2 Sauerkraut Fish and Lamb Pot': 24.95,
 'H3 Country Style Blood Curd Dish': 14.95,
 'H4 Beef Pot': 13.95,
 'H5 and Crispy Chicken': 12.95,
 'H6 Duck Stew': 12.95,
 'H7 Konjac Beef': 12.95,
 'H8 Lotus Tofu': 11.95,
 'H9 Three Pepper Beaten Duck': 13.95,
 'H10 Tea Smoked Duck': 13.95,
 'H11 Four Joy Lion Head': 12.95,
 'H12 Fragrant Ribs': 12.95,
 'F1 Northern Boiling Fish': 20.95,
 'F2 Sauerkraut Fish Pot': 20.95,
 'F3 Chengdu Style Pot Roasted Tilapia': 18.95,
 'F4 Mala Pot Roasted Tilapia': 18.95,
 'F5 Bean Curd Tilapia Stew': 18.95,
 'F7 Pot Roasted Black Bean Tilapia': 18.95,
 'F12 Steamed Tilapia': 18.95,
 'F9 Sweet and Sour Crispy Tilapia': 18.95,
 'F10 Scallion Pot Roasted Tilapia': 18.95,
 'B1 Water Boiled Beef': 13.95,
 'B2 Cumin Beef': 13.95,
 'B3 Black Pepper Beef': 13.95,
 'B4 Mala Beef Tofu': 13.95,
 'B5 Mongolian Beef': 13.95,
 'B6 Crispy Mala Beef': 13.95,
 'B7 Baby Pepper Black Bean Beef': 13.95,
 'B8 Baby Pepper and Shredded B

# IMPORT RESTERAUNT REVIEWS

## IMPORT YELP REVIEWS

In [11]:

reviews_df=pd.DataFrame()
  
user_agent = 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_14_5) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/13.0 Safari/605.1.15'
webpage = ''
nextpage = 'something'
xpath_reviews = '//script[@type="application/ld+json"]/text()'
xpath_nextpage = '//link[@rel="next"]'
headers = {'User-Agent': user_agent}

In [12]:
if food:
    reviewdf = pd.read_json('menu_parse/bigmala')
elif not food:
    reviewdf = pd.read_json('menu_parse/sharetea')
    s = lambda x: list(x.values())[0]
    reviewdf['reviewRating'] = reviewdf['reviewRating'].apply(s)
else:
    reviewdf = pd.DataFrame()

    while len(nextpage) > 0:
        #print('1')
        page = requests.get(webpage, headers = headers)#,proxies={"http": proxy, "https": proxy})
        print('pagegot','\r')
        parser = html.fromstring(page.content)
        reviews = parser.xpath(xpath_reviews)
        nextpage = parser.xpath(xpath_nextpage)

        if len(nextpage) > 0:
            nextlink = nextpage[0].get('href')
            print(nextlink, end = '\r')
            webpage = nextlink
            y = json.loads(list(reviews)[0])['review']

            reviewdict = {y.index(i):i for i in y}

            reviewdf = pd.concat([reviewdf,pd.DataFrame(reviewdict).T])
            
    s = lambda x: list(x.values())[0]
    reviewdf['reviewRating'] = reviewdf['reviewRating'].apply(s)

## IMPORT GOOGLE REVIEWS

#### todo: make place id google maps api modular through api call

In [13]:
import googlemaps
from datetime import datetime
import requests

In [14]:
gmapsapikey = json.load(open('menu_parse/gmapsapikey.json', 'r'))[0]

In [15]:
url = f'https://maps.googleapis.com/maps/api/place/details/json?key={gmapsapikey}&place_id={pid}&fields=name,review'
response = requests.get(url, headers=headers)


In [16]:
googlereviewdict = json.loads(response.content.decode('utf-8'))['result']['reviews']
googlereviewdict = {googlereviewdict.index(i):i for i in googlereviewdict}

googlereviewdf = pd.DataFrame(googlereviewdict).T

In [17]:
googlereviewdf = googlereviewdf[['author_name','rating','text']]
googlereviewdf.columns = ['author','reviewRating','description']
googlereviewdf

,author,reviewRating,description
0,Tristan & Ashley Boyd,5,Hubby found Mala Sichuan on Yelp and sent it a...
1,Pat Patel,4,Very good traditional Chinese food. We definit...
2,Epiphany Animation Studios,3,It was alright. I think maybe they had an off ...
3,Na Li,4,"I come here once a while, it used to one of my..."
4,Russell Monroe,2,"This review is for the Finn Hall location, but..."


In [18]:
reviewdf = pd.concat([reviewdf,googlereviewdf],sort = False)

In [19]:
reviewdf = reviewdf.reset_index().drop(['index'],axis=1)

In [20]:
#reviewdf['reviewRating'] = reviewdf['reviewRating'].apply(lambda x:x['ratingValue'])
reviewdf['description'] = reviewdf['description'].apply(lambda x:x.replace('\n','').replace('-',' ').lower())
reviewdf['description'] = reviewdf['description'].apply(lambda x:' '.join([i for i in x.split(" ") if len(i)>2]))
translator = str.maketrans(string.punctuation, ' '*len(string.punctuation)) #map punctuation to space
reviewdf['description'] = reviewdf['description'].apply(lambda x:x.translate(translator)).apply(lambda x: ' '.join(x.split()))



# GRAB MENU ITEMS

In [21]:
menuitems = json.load(open('menuJSON/mala7.json'))
if food:
    menuitems = [x[3:].strip() for x in list(menuitems.keys())]
menitems = [x.lower() for x in menuitems]

## RANKING MENU ITEMS

### sanity check of menu items

In [22]:
menitems

['top notch pot the outlaws',
 'sauerkraut fish and lamb pot',
 'country style blood curd dish',
 'beef pot',
 'and crispy chicken',
 'duck stew',
 'konjac beef',
 'lotus tofu',
 'three pepper beaten duck',
 'tea smoked duck',
 'four joy lion head',
 'fragrant ribs',
 'northern boiling fish',
 'sauerkraut fish pot',
 'chengdu style pot roasted tilapia',
 'mala pot roasted tilapia',
 'bean curd tilapia stew',
 'pot roasted black bean tilapia',
 'steamed tilapia',
 'sweet and sour crispy tilapia',
 'scallion pot roasted tilapia',
 'water boiled beef',
 'cumin beef',
 'black pepper beef',
 'mala beef tofu',
 'mongolian beef',
 'crispy mala beef',
 'baby pepper black bean beef',
 'baby pepper and shredded beef',
 'satay beef',
 'sauteed lamb with veggies',
 'cumin lamb']

In [23]:
search = lambda x: [menitem for menitem in menitems if menitem[:math.floor(0.8*len(menitem))] in x]
reviewdf['containedMenuItems'] = reviewdf['description'].apply(search)

In [24]:
menitemstoreviews = {}
for menitem in menitems:
    menitemstoreviews[menitem] = [{},list(reviewdf[reviewdf['containedMenuItems'].map(lambda d: menitem in d)].index)]

In [25]:

def compute(itor,ref):
    #total number of 4 plus reviews
    tot4plusreviews = len(ref[ref['reviewRating'] >= 4])
    for item in itor:
        #print(item,'------',end = '')
        allratings = []
        fourpluscounter = 0
        twominuscounter = 0
        totalrev = len(itor[item][1])

        for rev in itor[item][1]:
            curstar = ref.iloc[rev]['reviewRating']
            allratings.append(curstar)
            if curstar >= 4:
                fourpluscounter += 1
            elif curstar <= 2:
                twominuscounter += 1
        if not totalrev == 0:
            itor[item][0]['extremerev'] = (fourpluscounter - twominuscounter) / (2 * totalrev)
        else:
            itor[item][0]['extremerev'] = 0
        itor[item][0]['percentageoftotalreviews'] = 8.5 * (fourpluscounter) / tot4plusreviews

        if not len(allratings) == 0:
            itor[item][0]['avgrev'] = sum(allratings)/(5 * len(allratings))
            #print('av rating',sum(allratings)/len(allratings))
        else:
            itor[item][0]['avgrev'] = 0
            #print('no reviews')
    return itor
menitemstoreviews = compute(menitemstoreviews,reviewdf)

In [26]:
itemratings = pd.DataFrame(menitemstoreviews).T
itemratings.columns = ['ratings','indexes']

itemratings['totalscore'] = itemratings['ratings'].apply(lambda x: sum(x.values()))

itemratings['totalscore'].sort_values(ascending = False)

water boiled beef                    2.678521
cumin beef                           2.092424
and crispy chicken                   1.673229
cumin lamb                           1.621688
mala pot roasted tilapia             1.574015
three pepper beaten duck             1.565230
steamed tilapia                      1.543966
chengdu style pot roasted tilapia    1.529310
mongolian beef                       1.525172
black pepper beef                    1.517241
sauerkraut fish pot                  1.514655
satay beef                           1.514655
baby pepper and shredded beef        1.514655
sweet and sour crispy tilapia        1.514655
four joy lion head                   1.477299
crispy mala beef                     1.472947
tea smoked duck                      1.465405
top notch pot the outlaws            1.458621
northern boiling fish                1.429310
mala beef tofu                       1.413218
country style blood curd dish        1.329310
fragrant ribs                     

In [27]:
itemratings

,ratings,indexes,totalscore
top notch pot the outlaws,"{'extremerev': 0.5, 'percentageoftotalreviews'...","[98, 131, 416, 515]",1.458621
sauerkraut fish and lamb pot,"{'extremerev': 0, 'percentageoftotalreviews': ...",[],0.000000
country style blood curd dish,"{'extremerev': 0.5, 'percentageoftotalreviews'...","[416, 438]",1.329310
beef pot,"{'extremerev': 0.14285714285714285, 'percentag...","[252, 353, 392, 502, 554, 605, 714]",0.858621
and crispy chicken,"{'extremerev': 0.3918918918918919, 'percentage...","[11, 29, 38, 48, 66, 94, 101, 102, 104, 113, 1...",1.673229
duck stew,"{'extremerev': 0.16666666666666666, 'percentag...","[162, 214, 625]",0.929310
konjac beef,"{'extremerev': 0, 'percentageoftotalreviews': ...",[],0.000000
lotus tofu,"{'extremerev': 0.375, 'percentageoftotalreview...","[64, 88, 180, 318]",1.168966
three pepper beaten duck,"{'extremerev': 0.5, 'percentageoftotalreviews'...","[103, 349, 486, 539, 628, 653, 655, 671, 739]",1.565230
tea smoked duck,"{'extremerev': 0.3695652173913043, 'percentage...","[31, 42, 64, 113, 116, 172, 212, 227, 237, 257...",1.465405


# WIP / TESTING

In [28]:
# itemratings.loc['water boiled beef']['ratings']

# reviewdf[reviewdf['description'].map(lambda d: 'okinawa pearl milk tea' in d)]



# reviewdf.iloc[548]['description']



# pd.DataFrame(menitemstoreviews).T

# reviewdf[reviewdf['author']=='Tristan & Ashley Boyd']

# reviewdf.iloc[473]['description']







# #reviewdf.to_json('bigmala')

# USER PREFERENCES

## IMPORT USER REVIEWS

In [29]:
userpage = 'https://www.yelp.com/user_details_reviews_self?rec_pagestart=0&userid=2fKJeKlPi9le_ta7DPVW_A'
def wikiscrape(link):
    unextpage = 'something'
    userreviewdf = pd.DataFrame()

    upath_reviews = '//p[@lang="en"]//node()'
    upath_nextpage = '//a[@class="u-decoration-none next pagination-links_anchor"]'
    upath_starrating = '//div[@class="biz-rating__stars"]//div'

    while len(unextpage) > 0:

        userpage = requests.get(userpage, headers = headers)#,proxies={"http": proxy, "https": proxy})

        uparser = html.fromstring(userpage.content)

        ureviews = uparser.xpath(upath_reviews)

        unextpage = uparser.xpath(upath_nextpage)

        if len(unextpage) > 0:
            print(unextpage)
            userpage = unextpage[0].get('href')
        ustarratingpath = uparser.xpath(upath_starrating)

        ureviews = [str(i) if not "Element br" in str(i) else 0 for i in ureviews ]

        ufullreviews = grabfullreviws(ureviews)

        fullstars = [i.get('title') for i in ustarratingpath]

        userreviewdf = pd.concat([userreviewdf,pd.DataFrame(list(zip(fullstars,ufullreviews)))])
#userreviewdf = wikiscrape(username)

In [30]:
#userreviewdf = userreviewdf.reset_index().drop(['index'],axis = 1)

userreviewdf = pd.read_json('userreviews')

In [31]:
userreviewdf.columns = ['reviewRating','description']
userreviewdf['reviewRating'] = userreviewdf['reviewRating'].apply(lambda x: int(x[0:1]))
reviewdf['description'] = reviewdf['description'].apply(lambda x:x.replace('\n','').replace('-',' ').lower())
reviewdf['description'] = reviewdf['description'].apply(lambda x:' '.join([i for i in x.split(" ") if len(i)>2]))
translator = str.maketrans(string.punctuation, ' '*len(string.punctuation)) #map punctuation to space
reviewdf['description'] = reviewdf['description'].apply(lambda x:x.translate(translator)).apply(lambda x: ' '.join(x.split()))




In [32]:
userreviewdf.head()

,reviewRating,description
0,4,Leave a yelp review for free boba
1,5,TL;DR: An affordable apartment complex with gr...
10,5,I'm writing this review because I noticed that...
11,2,TLDR: Over rated Mediterranean cuisine with de...
12,3,I came in here initially about to order a burg...


## scrape wikipedia for chinese foods (only run once, save results as JSON)

In [33]:
from bs4 import BeautifulSoup

In [34]:

wikipage = 'https://en.wikipedia.org/wiki/List_of_Chinese_dishes'
wikipage = requests.get(wikipage).text
wikidf = pd.DataFrame()

soup = BeautifulSoup(wikipage)

my_tables = soup.findAll('table',{'class':'wikitable'})


In [35]:
chinesefoods = []
for table in my_tables:
    alla = list(table.findAll('a'))
    chinesefoods = chinesefoods +[x for x in[alla[i].get('title') for i in range(len(alla))] if type(x) == str]

In [36]:
from nltk.corpus import wordnet as wn
allfoods = list(set([w for s in wn.synset('food.n.02').closure(lambda s:s.hyponyms()) for w in s.lemma_names()])) + chinesefoods

In [37]:
allfoods = [x.lower() for x in allfoods]

In [51]:
with open('allfoods.json', 'w') as f:
    json.dump(allfoods, f)

## food preferences by users reviews

In [39]:
search = lambda x: [food for food in allfoods if food in x]

userreviewdf['containedFood'] = userreviewdf['description'].apply(search)

In [40]:
userreviewdf.head()

,reviewRating,description,containedFood
0,4,Leave a yelp review for free boba,[]
1,5,TL;DR: An affordable apartment complex with gr...,"[leg, hen, side, cookie, tuna]"
10,5,I'm writing this review because I noticed that...,"[lemon, chicken, delicious]"
11,2,TLDR: Over rated Mediterranean cuisine with de...,"[cos, hare, roe, hen, chicken, side, tuna, pit..."
12,3,I came in here initially about to order a burg...,"[velveeta, cheese, joint, hen, patty, side, av..."


In [41]:
foodtoreviewunfiltered = {}
for food in allfoods:
    foodtoreviewunfiltered[food] = [{},list(userreviewdf[userreviewdf['containedFood'].map(lambda d: food in d)].index)]

In [42]:
foodtoreview = {k:v for (k,v) in foodtoreviewunfiltered.items() if not len(v[1]) == 0}

In [43]:
userrevscores = compute(foodtoreview,userreviewdf)

In [44]:
userratings = pd.DataFrame(userrevscores).T
userratings.columns = ['ratings','indexes']

userratings['totalscore'] = userratings['ratings'].apply(lambda x: sum(x.values()))

userratings['totalscore'].sort_values(ascending = False).head()

hen     6.673237
meat    5.501807
side    5.435801
beef    4.733914
cut     4.200232
Name: totalscore, dtype: float64

In [45]:
for i in list(itemratings.index):
    test = i.split(" ")
    test = [x.lower() for x in test]
#     print(i)
    for j in test:
        if j in allfoods:
#             print ('before',itemratings.loc[i]['totalscore'])
            itemratings.at[i,'totalscore'] = itemratings.loc[i]['totalscore'] + 0.3 * userratings['totalscore'][test.index(j)]
#             print ('after',itemratings.at[i,'totalscore'])
#             print('\n')

In [46]:
itemratings['totalscore']

top notch pot the outlaws            1.458621
sauerkraut fish and lamb pot         0.542195
country style blood curd dish        1.329310
beef pot                             1.360206
and crispy chicken                   2.217620
duck stew                            1.430896
konjac beef                          0.090000
lotus tofu                           1.168966
three pepper beaten duck             2.107425
tea smoked duck                      2.009795
four joy lion head                   1.477299
fragrant ribs                        1.268966
northern boiling fish                1.973701
sauerkraut fish pot                  1.604655
chengdu style pot roasted tilapia    1.529310
mala pot roasted tilapia             1.574015
bean curd tilapia stew               0.501585
pot roasted black bean tilapia       0.452195
steamed tilapia                      1.543966
sweet and sour crispy tilapia        1.514655
scallion pot roasted tilapia         0.501585
water boiled beef                 

In [47]:

min_max_scaler = preprocessing.MinMaxScaler()

x = itemratings[['totalscore']].values.astype(float)
# Create an object to transform the data to fit minmax processor
itemrating_scaled = min_max_scaler.fit_transform(x)

# Run the normalizer on the dataframe

In [48]:
import math

In [49]:
itemratings['totalscore'] = [math.ceil(10* x[0]) for x in itemrating_scaled]

In [50]:
itemratings['totalscore']

top notch pot the outlaws             5
sauerkraut fish and lamb pot          2
country style blood curd dish         4
beef pot                              5
and crispy chicken                    7
duck stew                             5
konjac beef                           0
lotus tofu                            4
three pepper beaten duck              7
tea smoked duck                       7
four joy lion head                    5
fragrant ribs                         4
northern boiling fish                 7
sauerkraut fish pot                   5
chengdu style pot roasted tilapia     5
mala pot roasted tilapia              5
bean curd tilapia stew                2
pot roasted black bean tilapia        2
steamed tilapia                       5
sweet and sour crispy tilapia         5
scallion pot roasted tilapia          2
water boiled beef                    10
cumin beef                            7
black pepper beef                     7
mala beef tofu                        5


In [61]:
pref = open('foodchoice.txt', "r")  # load user preferences txt
pref_lines = pref.readlines()
budget = float(pref_lines[0].strip())
# budget = 100.0
eats_meat = bool(pref_lines[1].strip())
user_likes = (pref_lines[2].strip().split())
print(budget,type(budget),'budget')
print(eats_meat,type(eats_meat),'eats_meat')
print(user_likes,type(user_likes),'user_likes')

100.0 <class 'float'> budget
True <class 'bool'> eats_meat
['chili,', 'peanut'] <class 'list'> user_likes


In [62]:
pref = json.load(open('preferencesData.json', "r"))


budget = float(pref['budget'].strip())
# budget = 100.0
eats_meat = bool(pref['diet-veg'].strip())
eats_meat = not eats_meat
user_likes = (pref['diet-exclude'].strip().split())

print(budget,type(budget),'budget')
print(eats_meat,type(eats_meat),'eats_meat')
print(user_likes,type(user_likes),'user_likes')

1.0 <class 'float'> budget
False <class 'bool'> eats_meat
['chili,', 'peanut'] <class 'list'> user_likes
